# SenMo Net

concepts:
1. network
2. layer
3. neuron
4. connection

example: 

froward network:
layer a predicts b, c, and d, with decreasing accuracy
layer b predicts c, and d
layer c predicts d
layer d predicts nothing

backwards network:
layer d predicts c, b, and a, with decreasing accuracy
layer c predicts b, and a
layer b predicts a
layer a predicts nothing

forwards network competes against backwards network for the highest overall score of predictions. so, layer a in forward network competes against d in backwards network when it predicts d and d predicts a, whoever loses gets harsher errors applied. a also competes against b when they predict each other.

predictions are applied in time. a gets activated and produces predictions for all subsequent layers at t=0, but its prediction for b layer is applied right away and checked at t=1, whereas its prediction for c layer is applied at t=1 and checked at t=2, etc.

the backwards network is what trains the forward network, they replace back propogation for each other.

to be used in a sensorimotor context - an autonomous agent exploring an environment. forward network can be thought of as the sensory network, "given what I see (layer a), what motor neurons will fire (layer d)?". backwards network can be seen as the motor network, "given what motor neurons fire (layer d), what will I see (layer a)?"

this is the simplest form of the idea. for more efficiency shape as an autoencoder. for further efficiencies combine hierarchically.

In [ ]:
class Neuron():
    def __init__(self, neurons):
        self.neurons 
        